# Private network - SQL server
This template sets up a private SQL Server with a private endpoint connection. Please ensure an Azure Private DNS Zone is created before proceeding with the deployment, see [/scenario-private-dns-zones.ipynb](../private-dns-zones/scenario-private-dns-zones.ipynb).

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- A private DNS zone, see [/scenario-private-dns-zones.ipynb](../scenario-private-dns-zones/private-dns-zones.ipynb)


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [ ]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=northeurope

2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "private-sql-deployment"
template_file = "main.bicep"
parameters_file = "main.bicepparam"

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

4. Create an Azure Resource group:

In [ ]:
!az login
!az account set --subscription  {subscription_id}
!az group create --name {resource_group} --location {location}

## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [ ]:
%%writefile {parameters_file}

using 'main.bicep'

param sqlServerName = '${take('prvtsql${uniqueString(vnetRsourceGroup)}', 15)}'
param sqlAdministratorLogin = 'testsqladmin' // This is a placeholder value and should be tokenized with a secure value
param sqlAdministratorLoginPassword = 'Th3B3stPa$$word!' // This is a placeholder value and should be tokenized with a secure value

param databases = [
  {
    name: 'testlennartdb'
    sku: {
      name: 'Basic'
      tier: 'Basic'
      capacity: 5
    }
    collation: 'SQL_Latin1_General_CP1_CI_AS'
    maxSizeBytes: 104857600
    sampleName: 'AdventureWorksLT'
  }
]

param vnetRsourceGroup = 'my-private-network'
param vnetName = 'private-network-test'
param subnetName = 'default'


2. Deploy the template:

In [ ]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name} 


3. Optional: Login to the bastion jumpbox and test the connection.

    - Login to the VM:
![bastion-login.png](../../../images/private-network-docs/bastion-login.png)
    - Get the SQL server name:

In [ ]:
import os

sql_server_fqdn = os.popen(f"az deployment group show --resource-group {resource_group} --name {deployment_name} --query \"properties.outputs.sqlServerFqdn.value\" --output tsv").read().strip()

print(f"SQL server name: {sql_server_fqdn}")

    - Connect in the VM with the SQL Server Management Studio (SSMS) to the server
![sql-server-login.png](../../../images/private-network-docs/sql-server-login.png)
    
    - Explore! 
![sql-server-object-explorer.png](../../../images/private-network-docs/sql-server-object-explorer.png)